In [1]:
import pandas as pd
import numpy as np
import json
import re
import glob
from datetime import date
import math

from scipy.stats import rankdata
import sys

main_dir = '/Users/matty_gee/Desktop/SNT/SNT-behavior'
sys.path.insert(0, main_dir + '/code')
# from code_base import *
from data_prep_utils import * 
from os.path import exists
from compute_task_variables import *

# dumb to have different orders everywhere...should just be explicit...
img_sets = {'OFA': ['OlderFemaleBl_2','OlderMaleW_1','OlderMaleW_4','OlderFemaleBr_3','OlderMaleBr_2','OlderFemaleW_1'],
            'OFB': ['OlderFemaleW_2','OlderMaleBr_1','OlderMaleBl_3','OlderFemaleW_3','OlderMaleW_5','OlderFemaleBl_1'],
            'OFC': ['OlderFemaleBl_2','OlderMaleBr_1','OlderMaleW_5','OlderFemaleW_3','OlderMaleBr_4','OlderFemaleW_1'],
            'OFD': ['OlderFemaleW_2','OlderMaleW_1','OlderMaleBr_3','OlderFemaleBr_3','OlderMaleW_5','OlderFemaleBl_1'],
            'OMA': ['OlderMaleBr_2','OlderFemaleW_2','OlderFemaleW_3','OlderMaleBr_1','OlderFemaleBr_5','OlderMaleW_5'],
            'OMB': ['OlderMaleW_1','OlderFemaleBl_2','OlderFemaleBl_3','OlderMaleW_4','OlderFemaleW_1','OlderMaleBr_4'], 
            'OMC': ['OlderMaleBr_4','OlderFemaleBl_2','OlderFemaleW_3','OlderMaleW_3','OlderFemaleBl_1','OlderMaleW_5'],
            'OMD': ['OlderMaleW_1','OlderFemaleW_2','OlderFemaleBr_5','OlderMaleBr_3','OlderFemaleW_1','OlderMaleBr_4'],
            'YFA': ['YoungerFemaleBr_1','YoungerMaleW_4','YoungerMaleW_3','OlderFemaleBr_5','OlderMaleBr_4','OlderFemaleW_1'],
            'YFB': ['YoungerFemaleW_3','YoungerMaleBr_2','OlderMaleBr_3','OlderFemaleW_4','YoungerMaleW_2','OlderFemaleBl_1'],
            'YFC': ['YoungerFemaleBr_1','YoungerMaleBr_2','OlderMaleW_4','OlderFemaleW_3','OlderMaleBr_4','OlderFemaleW_1'],
            'YFD': ['YoungerFemaleW_3','YoungerMaleW_4','OlderMaleBr_3','OlderFemaleBr_5','OlderMaleW_5','OlderFemaleBl_1'],
            'YMA': ['YoungerMaleBr_2','YoungerFemaleW_3','OlderFemaleW_4','OlderMaleBr_3','OlderFemaleBl_1','YoungerMaleW_2'],
            'YMB': ['YoungerMaleW_4','YoungerFemaleBr_1','OlderFemaleBr_5','YoungerMaleW_3','OlderFemaleW_1','OlderMaleBr_4'],
            'YMC': ['YoungerMaleBr_2','YoungerFemaleBr_1','OlderFemaleW_3','OlderMaleW_3','OlderFemaleBl_1','OlderMaleW_5'],
            'YMD': ['YoungerMaleW_4','YoungerFemaleW_3','OlderFemaleBr_3','OlderMaleBr_3','OlderFemaleW_1','OlderMaleBr_4']}

# info about the validted decisions: e.g., the continuous ratings etc..
decisions_info = pd.read_excel(main_dir + '/Online/Prolific/Info/TaskValidation_Val20_SortedOptionMeans_Mem50_n81.xlsx')
decisions_info = decisions_info.sort_values(by = 'Decision').reset_index(drop=True)
character_roles = ['first', 'second', 'assistant', 'powerful', 'boss', 'neutral'] # order in matlab

data_dir = main_dir + '/Online/Prolific/Data/Replication_2022'
pavlovia_dir = '/Users/matthew/Desktop/social-navigation/data'
repl_data_dir = data_dir + '/Task/Raw-Pavlovia'

//anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


# Get pavlovia files

In [180]:
import shutil
for file in glob.glob(pavlovia_dir + "/*.csv"):
    try:
        data = pd.read_csv(file)
        sub  = data['prolific_id'].values[0]
        exp  = data['experiment'].values[0]
        if (sub != '6168883c6837227155b2360e') & (isinstance(sub, str)): # avoid my prolific id (but there are other ones too) & ones without prolific ids
            if exp == 'behavioral':
                if '2022-02' in file:
                    cp_file = repl_data_dir + '/' + file.split('/')[-1]   
            if not exists(cp_file):
                shutil.copyfile(file, cp_file)
    except:
        continue

In [11]:
sub_files = [sub_data for sub_data in glob.glob(repl_data_dir + "/*.csv")]
print(len(sub_files))

missing = []
posttask_dfs = []
dates = []

for file in sub_files:
    data = pd.read_csv(file)
    sub_id = data['prolific_id'].values[0]
    if sub_id not in ['5fb28e64134ec0894396633a', '60393e9ddde61e3a5463746d', '5fb28e64134ec0894396633a']:
        if data.shape[0] > 1: data = pd.DataFrame(data.iloc[0,:]).T # sometimes saves multiple times into diff rowss..?
        data.replace('newcomb','powerful', inplace=True)
        data.replace('hayworth','boss', inplace=True)
        data.columns = data.columns.str.replace("newcomb", "powerful")
        data.columns = data.columns.str.replace("hayworth", "boss")
        
        date = file.split('/')[-1].split('_')[3]        
        
        ############################################################
        # narrative task
        ############################################################

        narrative_fname = data_dir + '/Task/Organized/SNT_' + sub_id + '.xlsx'
        if not exists(narrative_fname):
            try: 
                
                narrative_resp = np.array([int(re.sub('["\]"]', '', d.split(':')[1])) for d in data['narrative.choices'].values[0].split(',')])
                narrative_opt_order = data['narrative.opts_order'].values[0].split('","')
                narrative_resp_data = np.zeros(len(narrative_resp))
                affl = []
                pwr  = []
                for q, question in enumerate(narrative_opt_order):
                    question = ''.join(filter(lambda item: not item.isdigit(), question))
                    for sep in [';', ',"', '.,']:
                        if sep in question:
                            opt1 = remove_nontext(question.split(sep)[1])
                            opt2 = remove_nontext(question.split(sep)[2])
                    sort_ix = np.argsort((opt1,opt2)) # order options alphabetically
                    resp = sort_ix[narrative_resp[q]-1]+1 # choice -> 1 or 2, depending on alphabetical ordering
                    narrative_resp_data[q] = resp
                    decision = decisions_info.iloc[q]
                    affl.append(decision['Opt{}_Affl_Discrete'.format(int(resp))])
                    pwr.append(decision['Opt{}_Pwr_Discrete'.format(int(resp))])
                narrative_rts = np.array([int(remove_nontext(rt)) for rt in data['narrative.rts'].values[0].split(',')])
                decision_rts  = narrative_rts[np.array(decisions_info['Slide_num'])-1]
                narrative_df = pd.DataFrame((narrative_resp, affl, pwr, decision_rts)).T
                narrative_df.columns = ['button_press', 'affil', 'power', 'rt']
                narrative_df.insert(0, 'trial_num', np.arange(1,64))
                narrative_df.to_excel(narrative_fname, index=False)

            except:
                # if no task, just skip
                continue
            
        ############################################################
        # memory
        ############################################################

        try: 
            
            mem_cols = [c for c in data.columns if 'memory' in c]
            mem_ques = data[[c for c in mem_cols if 'question' in c]].values[0]
            mem_resp = data[[c for c in mem_cols if 'resp' in c]].values[0]
            mem_rt = data[[c for c in mem_cols if 'rt' in c]].values[0]
            mem_data = sorted(list(zip(mem_ques, mem_resp)))

            # summary 
            mem_correct = [1,4,5,4,5,0,0,0,4,1,3,3,1,4,5,3,1,0,2,5,5,2,2,2,3,3,0,1,2,4] # alphabetical for memry qs, according to the order of matlab outputs
            mem_acc = np.zeros(6) 
            for r,resp in enumerate(mem_data): 
                if resp[1] == character_roles[mem_correct[r]]:
                    mem_acc[mem_correct[r]]+=1
            mem_acc_df = pd.DataFrame((mem_acc/5).reshape(1,-1), columns=['memory_' + char for char in character_roles])
            mem_acc_df.insert(6, 'memory_mean', np.mean(mem_acc_df.values))
            mem_acc_df.insert(6, 'memory_rt', np.mean(mem_rt))

            # trial x trial
            mem_questions = ['memory_' + str(q+1) + '_'  + character_roles[r] for q,r in enumerate(mem_correct)]
            mem_responses = [character_roles[character_roles.index(resp[1])] for r,resp in enumerate(mem_data)] # find the role of the character response
            mem_resp_df = pd.DataFrame(np.array(mem_responses).reshape(1,-1), columns=mem_questions)

            mem_df = pd.concat([mem_acc_df, mem_resp_df],1)
            
        except: 
            # if no memory just skip
            continue

        ############################################################
        # demos of the characters 
        ############################################################

        try: 
            
            char_skincolor = []
            char_gender = []
            for char in character_roles: 
                img = data['character_info.' + char + '.img'].values[0]
                if 'Br_' in img: char_skincolor.append('brown')
                else: char_skincolor.append('white')
                if 'Male' in img: char_gender.append('man')
                else: char_gender.append('woman')
            gender_df = pd.DataFrame(np.array(char_gender).reshape(1,-1), columns=['gender_' + role for role in character_roles])
            skincolor_df = pd.DataFrame(np.array(char_skincolor).reshape(1,-1), columns=['skincolor_' + role for role in character_roles])
            
        except:
            continue
                
        ############################################################
        # post-task
        ############################################################

        try: 
            
            ############
            # demo: get mean
            ############
            
            demo_df = get_and_rename(data, 'demographics')
            dims = ['masculine', 'feminine', 'youthful']
            for dim in dims: 
                mean_rating = np.mean(demo_df[[c for c in demo_df.columns if dim + '.resp' in c]],1).values[0]
                mean_rt = np.mean(demo_df[[c for c in demo_df.columns if dim + '.rt' in c]],1).values[0]
                demo_df.insert(0, 'mean.' + dim, mean_rating)
                demo_df.insert(1, 'mean.' + dim + '.rt', mean_rt)

            ############
            # dots: normalize & get mean
            ############
            
            dots_df = data[[c for c in data.columns if 'dots.' in c]]
            for col in dots_df.columns: # rename
                split_col = col.split('.')
                rename_col = split_col[0] + '_' + split_col[2] + '_' + split_col[1]
                dots_df.rename(columns={col:rename_col}, inplace=True)
                
            dots_affil = (dots_df[[c for c in dots_df.columns if ('affil' in c) & ('redos' not in c)]] - 500)/500
            dots_affil.insert(0, 'dots_affil_mean', np.mean(dots_affil, 1))
            dots_power = (500 - dots_df[[c for c in dots_df.columns if ('power' in c) & ('redos' not in c) & ('affil' not in c)]])/500
            dots_power.insert(0, 'dots_power_mean', np.mean(dots_power, 1))
            dots_df = pd.concat([dots_affil, dots_power], 1)

            ############
            # judgments: get mean
            ############
            
            judgments_df = get_and_rename(data, 'judgments')
            dims = ['similarity', 'friendliness', 'competence', 'popularity', 'dominance', 
                    'likability', 'likability_self', 'impact', # unfortunately did not collect impact on self ratings yet...
                    'intensity', 'stability']
            for dim in dims: 
                mean_rating = np.mean(judgments_df[[c for c in judgments_df.columns if (dim + '.resp' in c) & ('self_self' not in c)]],1).values[0]
                mean_rt = np.mean(judgments_df[[c for c in judgments_df.columns if (dim + '.rt' in c) & ('self_self' not in c)]],1).values[0]
                judgments_df.insert(0, 'mean.' + dim + '.resp', mean_rating)
                judgments_df.insert(1, 'mean.' + dim + '.rt', mean_rt)

                
                
            ############
            # choice data - organize
            ############
            
            comparisons = [['assistant','first'], ['assistant','hayworth'], ['assistant','neutral'], ['assistant','newcomb'], ['assistant','second'], 
                          ['first','hayworth'], ['first','neutral'], ['first','newcomb'], ['first','second'],
                          ['hayworth','neutral'], ['hayworth','newcomb'], ['hayworth','second'], ['neutral','newcomb'], ['neutral','second'], ['newcomb','second']]
            columns = []
            for c,comp in enumerate(comparisons):
                columns.append('forced_choice_' + str(c+1) + '_' + comp[0])
                columns.append('forced_choice_' + str(c+1) + '_' + comp[1])
                columns.append('forced_choice_' + str(c+1) + '_side')
            choice_df = pd.DataFrame(columns=columns)

            try:
                
                choice_df_ = data[[c for c in data.columns if 'forced_choice' in c]]
                choice_data = []
                choice_cols = []
                for t in np.arange(0,15):
                    options = choice_df_['forced_choice.' + str(t) + '.comparison'].values[0].split('_&_')
                    resp = choice_df_['forced_choice.' + str(t) + '.resp'].values[0] - 50
                    if resp < 0: 
                        side = 'L'
                        choice = options[0]
                    else: 
                        side = 'R' 
                        choice = options[1]
                    ans = np.array([0,0])
                    sorted_ = sorted(options)
                    ans_ix = sorted_.index(choice)
                    ans[ans_ix] = np.abs(resp)
                    ans[np.abs(ans_ix-1)] = -np.abs(resp)

                    choice_cols.extend(['forced_choice.' + str(t) + '_' + sorted_[0], 'forced_choice.' + str(t) + '_' + sorted_[1], 'forced_choice.' + str(t) + '_side'])
                    choice_data.append([('+').join(sorted_)] + list(ans) + list(side))

                sort_df = pd.DataFrame(np.array(choice_data))
                sort_df = sort_df.sort_values(0, ascending=True).reset_index(drop=True)
                choice_df.loc[0,:] = sort_df.values[:,1:].flatten()
                
            except: 
                pass
                
            choice_df.columns = choice_df.columns.str.replace("newcomb", "powerful")
            choice_df.columns = choice_df.columns.str.replace("hayworth", "boss")
            for char in character_roles:
                choice_df['forced_choice_mean_' + char] = np.mean(choice_df[[c for c in choice_df.columns if char in c]].values.astype(int))
            
            ############
            # others - no changes
            ############
            
            race_df = data[[c for c in data.columns if 'race' in c]]
            free_resp_df = data[[c for c in data.columns if 'character_free_responses' in c]]
            story_ques_df = data[[c for c in data.columns if 'storyline_questions' in c]] 
            beh_ques_df = data[[c for c in data.columns if 'behavior_questions' in c]] 

            post_df = pd.concat([mem_df, gender_df, skincolor_df, demo_df, race_df, dots_df, judgments_df, choice_df, free_resp_df, story_ques_df, beh_ques_df], axis=1)

            # rename columns
            dims = ['youthful', 'feminine', 'masculine', 'race', 'stability', 'intensity', 
                    'impact', 'likability', 'dominance', 'competence', 'friendliness', 'similarity', 'popularity']
            for dim in dims: 
                col_names = [c for c in post_df.columns if dim in c]
                for col in col_names:
                    split_name = col.split('.')
                    rename = split_name[1] + '_' + split_name[0]
                    if 'rt' in col: rename = rename + '_rt'
                    post_df.rename(columns={col:rename}, inplace=True)

            ############
            # iq
            ############      

            iq_ques = [q.split('.')[1] for q in [c for c in data.columns if ('iq' in c) & ('resp' in c)]]
            iq_resp = data[[c for c in data.columns if ('iq' in c) & ('resp' in c)]].values[0]
            iq_rt = data[[c for c in data.columns if ('iq' in c) & ('rt' in c)]].values[0]

            answers =  [["VR4",5],["VR16","Its"],["VR17",47],["VR19","Sunday"],
                        ["LN7","X"],["LN33","G"],["LN34","X"],["LN58","N"],
                        ["MX45","E"],["MX46","B"],["MX47","B"],["MX55","D"],
                        ["R3D3","C"],["R3D4","B"],["R3D6","F"],["R3D8","G"]]
            iq_correct = np.zeros(len(answers))
            for answer in answers: 
                for (q,ques), resp in zip(enumerate(iq_ques), iq_resp):
                    if (ques == answer[0]) & (resp == answer[1]):
                        iq_correct[q] = 1

            post_df.insert(0, 'iq_score', np.mean(iq_correct))
            
        except:
            
            missing.append(['posttask', file.split('/')[-1]])
            pass

        ############################################################
        # realworld
        ############################################################  

        # sni stuff
        num_ppl = []
        network_div = []
         
        # enforce a dating column - not everyone will have
        categories = ['Marriage','Dating','Children','Parents','Inlaws','Relatives','Friends','Religion','School',
                      'Work','Work_supervision','Work_nonsupervision','Neighbors','Volunteer',
                      'Extra_group1', 'Extra_group2', 'Extra_group3', 'Extra_group4', 'Extra_group5'] 
        for cat in categories:
            try:
                resp = data['realworld_relationships.' + cat + '.number_of_people.value'].values[0]
                if cat == 'Dating':
                    if (resp == 1):
                        network_div.append(1)
                        num_ppl.append(1)
                    elif (resp == 3): 
                        network_div.append(1)
                        num_ppl.append(2)
                    elif (resp == 2) or (resp == 4): 
                        network_div.append(0)
                        num_ppl.append(0) 
                else:
                    if math.isnan(resp):
                        num_ppl.append(0)
                        network_div.append(0)
                    else:
                        num_ppl.append(resp)
                        if (resp != 0):
                            network_div.append(1)
                        else:
                            network_div.append(0)
            except: 
                num_ppl.append(0)
                network_div.append(0)

        sni_df = pd.DataFrame(np.array([np.sum(num_ppl), np.sum(network_div)]).reshape(1,-1), columns=['sni_number_ppl', 'sni_network_diversity'])
        initials_ = [c for c in data.columns if ('realworld' in c) & ('initials' in c)]
        realworld = []
        for initials in initials_:
            category = initials.split('.')[1]
            person_ = initials.split('.initials')[0]
            cols = ['time_known', 'frequency', 'similarity', 
                    'likability', 'impact', 
                    'popularity', 'competence', 'friendliness', 'dominance', 
                    'dots_affil', 'dots_power']
            # only allowed up to 10 in the dots... so might not be some columns even if there are initials
            try:
                resps = data[[person_ + '.' + col for col in cols]].values[0]
                realworld.append(list([category]) + list(resps))
            except:
                pass
            
        realworld_df = pd.DataFrame(realworld, columns=['realworld_category'] + ['realworld_' + col for col in cols])
        realworld_df['realworld_dots_affil'] = (realworld_df['realworld_dots_affil'] - 500)/500
        realworld_df['realworld_dots_power'] = (500 - realworld_df['realworld_dots_power'])/500
        realworld_data = realworld_df.values.flatten()
        columns = np.array([['realworld_category.' + str(i)] + ['realworld_' + col  + '.' + str(i) for col in cols] for i in np.arange(1,11)]).flatten()
        realworld_df = pd.DataFrame(columns=list(columns))
        realworld_df.loc[0,0:len(realworld_data)] = list(realworld_data)

        # put it all together
        posttask_df = pd.concat([post_df, sni_df, realworld_df],1)
        posttask_df.insert(0, 'sub_id', sub_id)
        posttask_df.insert(1, 'date', date)
        posttask_df.reset_index(drop=True, inplace=True)
        posttask_dfs.append(posttask_df)
        
posttask = pd.concat(posttask_dfs)
posttask.to_excel(data_dir + '/Summary/Posttask-replication_summary_n' + str(len(posttask)) + '.xlsx', index=False)


350


# Redcap
Dl Csvs first

In [22]:
questionnaire_csvs = glob.glob(data_dir + "/Questionnaires/*.csv")
csv_dfs = []
for csv in questionnaire_csvs: 
    
    raw_df = pd.read_csv(csv)
    raw_df = raw_df[[c for c in raw_df if 'complete' not in c and 'timestamp' not in c]]
    
    # demos
    demo_df = subset_df(raw_df, ['demo'])
    demo_df.rename(columns = {'demo_race___1': 'demo_race_amer_indian_or_alaska_native',
                             'demo_race___2': 'demo_race_asian',
                             'demo_race___3': 'demo_race_black/aa',
                             'demo_race___4': 'demo_race_latino/hispanic',
                             'demo_race___5': 'demo_race_multiracial',
                             'demo_race___6': 'demo_race_nat_hawaiian_or_pac_isl',
                             'demo_race___7': 'demo_race_white',
                             'demo_race___8': 'demo_race_other',
                             'demo_race_other': 'demo_race_other_text',
                             'demo_sex': 'demo_sex_1F',
                             'demo_gender': 'demo_gender_1W',
                             'demo_handedness': 'demo_handedness_1R'}, 
                    inplace = True)

    # oci
    oci_df = subset_df(raw_df, ['oci'])
    oci_df['oci_score'] = np.sum(oci_df, axis=1)
    oci_df['oci_att'] = raw_df['oci_att']

    # bapq
    bapq_df = score_bapq(raw_df) # includes items

    # apdis
    apdis_df = raw_df[['apdis_score', 'apdis_att'] + ['apdis_' + str(i) for i in np.arange(1,9)]]

    # lsas_av
    lsas_df = raw_df[['lsas_av_score', 'lsas_av_att'] + ['lsas_av_' + str(i) for i in np.arange(1,25)]]

    # zbpd
    zbpd_df = raw_df[['zbpd_score', 'zbpd_att'] + ['zbpd_' + str(i) for i in np.arange(1,11)]]

    # sds
    sds_df_ = subset_df(raw_df, ['sds'])
    sds_df = reverse_score(sds_df_, 4, sds_rev)
    sds_df['sds_score'] = np.sum(sds_df, axis=1)
    sds_df['sds_att'] = raw_df['sds_att']

    # aes
    # -- some are reversed (4,3,2,1): 1,2,3,4,5,7,8,9,12,13,14,15,16,17,18
    aes_raw = raw_df[['aes_' + str(s) for s in np.arange(1,19)]].values
    aes_raw[aes_raw == 3] = 4
    aes_raw[aes_raw == 2] = 3
    aes_raw[aes_raw == 0] = 1
    aes_df_ = pd.DataFrame(aes_raw, columns=raw_df[['aes_' + str(s) for s in np.arange(1,19)]].columns.values)

    aes_df  = reverse_score(aes_df_, 4, aes_rev)
    aes_df['aes_score'] = np.sum(aes_df, axis=1)
    aes_df['aes_cog_score'] = np.sum(aes_df.iloc[:,np.array([1,3,4,5,8,11,12,15])-1], axis=1)
    aes_df['aes_beh_score'] = np.sum(aes_df.iloc[:,np.array([2,6,9,10,12])-1], axis=1)
    aes_df['aes_emt_score'] = np.sum(aes_df.iloc[:,np.array([7,14])-1], axis=1)
    aes_df['aes_oth_score'] = np.sum(aes_df.iloc[:,np.array([15,17,18])-1], axis=1)
    aes_df['aes_att'] = raw_df['aes_att'] 

    # sss
    sss_df = score_sss(raw_df) # includes items

    # ipip
    ipip_rev = np.array([6,7,8,9,10,15,16,17,18,19,20])-1
    ipip_df = reverse_score(raw_df[['mini_ipip_' + str(s) for s in np.arange(1,21)]], 5, ipip_rev)
    extraversion = np.sum(ipip_df.iloc[:, np.array([1,6,11,16])-1],1)
    agreeableness = np.sum(ipip_df.iloc[:,np.array([2,7,12,17])-1],1)
    conscientiousness = np.sum(ipip_df.iloc[:,np.array([3,8,13,18])-1],1)
    neuroticism = np.sum(ipip_df.iloc[:,np.array([4,9,14,19])-1],1)
    intellect = np.sum(ipip_df.iloc[:,np.array([5,10,15,20])-1],1)
    ipip_df.insert(0, 'mini_ipip_extraversion', extraversion)
    ipip_df.insert(1, 'mini_ipip_agreeableness', agreeableness)
    ipip_df.insert(2, 'mini_ipip_conscientiousness', conscientiousness)
    ipip_df.insert(3, 'mini_ipip_neuroticism', neuroticism)
    ipip_df.insert(4, 'mini_ipip_intellect', intellect)

    # ius
    ius_df = raw_df[['ius_' + str(s) for s in np.arange(1,28)]]
    ius_1 = np.sum(ius_df.iloc[:,np.array([1,2,3,9,12,13,14,15,16,17,20,22,23,24,25])-1],1).values # f1 - negative behavioral and self-reverent implicationss
    ius_2 = np.sum(ius_df.iloc[:,np.array([4,5,6,7,8,10,11,18,19,21,26,27])-1],1).values # f2 - uncertainty is unfair 
    ius_df.insert(1, 'ius_f1', ius_1)
    ius_df.insert(2, 'ius_f2', ius_2)    
    ius_df.insert(0, 'ius_score', ius_1 + ius_2) 

    # put together
    csv_df = pd.concat([ipip_df, ius_df, sss_df, aes_df, demo_df, oci_df, bapq_df, apdis_df, lsas_df, zbpd_df, sds_df],1)
    csv_df.insert(0, 'sub_id', raw_df['prolific_pid'].values)
    csv_df.insert(1, 'sub_id_corrected', raw_df['prolific_pid_corrected'].values)
    csv_dfs.append(csv_df)
    
ques_df = pd.concat(csv_dfs)
ques_df.reset_index(inplace=True, drop=True)
ques_df.to_excel(data_dir + '/Summary/Questionnaire-replication_summary_n' + str(len(ques_df)) + '.xlsx', index=False)

/var/folders/jq/pk75wqp554g5dhplx1d3m91h0000gw/T/ipykernel_64815/1521642209.py:89: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  csv_df = pd.concat([ipip_df, ius_df, sss_df, aes_df, demo_df, oci_df, bapq_df, apdis_df, lsas_df, zbpd_df, sds_df],1)


# Behavior

In [2]:
# compute the behavioral geometry            
base_dir = main_dir + '/Online/Prolific/Data/Replication_2022'
task_files = glob.glob(base_dir + '/Task/Organized/*.xlsx')
compute_task_variables(base_dir, task_files)

# Combine

In [5]:
summary_dir = main_dir + '/Online/Prolific/Data/Replication_2022/Summary'
post_df = pd.read_excel(glob.glob(summary_dir + '/Posttask*')[0])
task_df = pd.read_excel(glob.glob(summary_dir + '/SNT*')[0])
survey_df = pd.read_excel(glob.glob(summary_dir + '/Questionnaire*')[0])
corrected = survey_df['sub_id_corrected'].values.astype(str)
for c, corr in enumerate(corrected):
    if corr != 'nan': survey_df['sub_id'][c] = corr.strip()

# find overlap
task_subs = task_df['sub_id'].values
post_subs = post_df['sub_id'].values
survey_subs = survey_df['sub_id'].values

overlap = list(set(survey_subs) & set(task_subs) & set(post_subs))
sub_dfs = []
for sub in overlap:
    
    task_ix = np.where(task_subs == sub)[0][0]
    post_ix = np.where(post_subs == sub)[0][0]
    survey_ix = np.where(survey_subs == sub)[0][0]
    sub_data = np.concatenate((task_df.iloc[task_ix, 1:], 
                               post_df.iloc[post_ix, 1:], 
                               survey_df.iloc[survey_ix, 1:])).reshape(1,-1)                      
    colnames = np.concatenate((task_df.iloc[:, 1:].columns.values, 
                               post_df.iloc[:, 1:].columns.values, 
                               survey_df.iloc[:, 1:].columns.values))
    sub_dfs.append(pd.DataFrame(sub_data, columns=colnames)) 
    
data_df = pd.concat(sub_dfs)
data_df.insert(0, 'sub_id', overlap)
data_df['demo_employed'] = ((data_df['demo_occupation']!=8) & (data_df['demo_occupation']!=9)) * 1    
data_df.to_excel(data_dir + '/Summary/All-data-replication_summary_n' + str(len(data_df)) + '.xlsx', index=False)